<a href="https://colab.research.google.com/github/ssuxmin/tmp/blob/main/automl_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import numpy as np
import random
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/범죄유형/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/범죄유형/test.csv')

In [20]:
print(sorted(train['범죄발생지'].unique()))
print(sorted(test['범죄발생지'].unique()))

['공원', '백화점', '병원', '식당', '약국', '은행', '인도', '주거지', '주유소', '주차장', '차도', '편의점', '학교', '호텔/모텔']
['공원', '백화점', '병원', '식당', '약국', '은행', '인도', '주거지', '주유소', '주차장', '차도', '편의점', '학교', '호텔/모텔']


# 시드(seed) 고정  
매번 고정된 결과를 얻기 위해서 사용합니다.  
시드를 고정하지 않는다면 같은 코드라도 매번 다른 결과가 나올 수 있습니다.

In [21]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [35]:
x_train = train.drop(['ID', 'TARGET'], axis = 1)
y_train = train['TARGET']

x_test = test.drop('ID', axis = 1)

In [23]:
x_train = train.drop(['ID'], axis = 1)
y_train = train['TARGET']

x_test = test.drop('ID', axis = 1)

In [36]:
ordinal_features = ['요일', '범죄발생지']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(x_train[feature])
    x_train[feature] = le.transform(x_train[feature])

    # x_train데이터에서 존재하지 않았던 값이 x_test 데이터에 존재할 수도 있습니다.
    # 따라서 x_test 데이터를 바로 변형시키지 않고 고윳값을 확인후 x_test 데이터를 변환합니다.
    for label in np.unique(x_test[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    x_test[feature] = le.transform(x_test[feature])

In [25]:
pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
from pycaret import classification
classification.setup(data=x_train,target='TARGET',
                 remove_outliers=True)

ValueError: ignored

In [28]:
best = classification.compare_models(n_select=5,fold=5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.5492,0.6990,0.5492,0.5460,0.5389,0.2884,0.2942,2.6160
gbc,Gradient Boosting Classifier,0.5477,0.6970,0.5477,0.5462,0.5373,0.2863,0.2924,24.7480
xgboost,Extreme Gradient Boosting,0.5435,0.6925,0.5435,0.5392,0.5335,0.2801,0.2853,18.6400
ada,Ada Boost Classifier,0.5428,0.6861,0.5428,0.5406,0.5333,0.2809,0.2859,2.7020
rf,Random Forest Classifier,0.5238,0.6739,0.5238,0.5168,0.5133,0.2491,0.2534,11.2540
et,Extra Trees Classifier,0.4997,0.6523,0.4997,0.4909,0.4900,0.2134,0.2163,11.7200
nb,Naive Bayes,0.4687,0.6106,0.4687,0.4645,0.4632,0.1761,0.1774,0.1680
lda,Linear Discriminant Analysis,0.4560,0.6001,0.4560,0.4178,0.3868,0.0935,0.1142,0.3320
ridge,Ridge Classifier,0.4543,0.0000,0.4543,0.3973,0.3703,0.0826,0.1072,0.1660
lr,Logistic Regression,0.4417,0.5730,0.4417,0.4187,0.3719,0.0673,0.0846,14.6940


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

In [38]:
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)


x_train shape: (84406, 18)
x_test shape: (17289, 18)


In [41]:
probas = []
for model in best:
    probas.append(model.predict_proba(x_test))

pred = [np.argmax(i) for i in np.array(probas).mean(axis=0)]

In [54]:
from lightgbm import LGBMClassifier

In [82]:
# LightGBM 모델 인스턴스화
lgbm_model = LGBMClassifier()

# 모델 학습
lgbm_model.fit(x_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [83]:
# 테스트 데이터에 대한 예측
pred = lgbm_model.predict(x_test)
pred

array([2, 0, 0, ..., 0, 0, 1])

In [86]:
# 제출 파일을 읽어옵니다.
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/범죄유형/sample_submission.csv') 

In [105]:
# 예측한 값을 TARGET 컬럼에 할당합니다.
submit['TARGET'] = pred
submit.head()

,ID,TARGET
0,TEST_00000,2
1,TEST_00001,0
2,TEST_00002,0
3,TEST_00003,0
4,TEST_00004,0


In [106]:
print(submit.head())


           ID  TARGET
0  TEST_00000       2
1  TEST_00001       0
2  TEST_00002       0
3  TEST_00003       0
4  TEST_00004       0


In [88]:
# 예측 결과 출력
print(predictions)

[2 0 0 ... 0 0 1]


In [100]:
# 'ID' 열을 포함하여 제출 파일을 다시 읽어옵니다.
sub = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/범죄유형/sample_submission.csv')

print(sub.columns)

Index(['ID', 'TARGET'], dtype='object')


In [101]:
# 예측한 결과를 파일로 저장합니다. index 인자의 값을 False로 설정하지 않으면 제출이 정상적으로 진행되지 않습니다.
sub.set_index('ID',inplace=True)
sub['TARGET'] = pred

In [102]:
# 'ID' 컬럼이 제대로 포함되어 있는지 확인합니다.
if 'ID' not in sub.columns:
    sub.insert(0, 'ID', sub.index)

In [103]:
print(sub.head())

                    ID  TARGET
ID                            
TEST_00000  TEST_00000       2
TEST_00001  TEST_00001       0
TEST_00002  TEST_00002       0
TEST_00003  TEST_00003       0
TEST_00004  TEST_00004       0


In [104]:
# 수정된 데이터프레임을 파일로 저장합니다.
sub.to_csv('./lightbgm_submit.csv', index=False)


# 다시 구현(dense.layer)


In [108]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [109]:
# Sequential 모델 인스턴스화
model = Sequential()

In [111]:
# Dense 레이어 추가
model.add(Dense(units=64, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))  # 이진 분류의 경우


In [112]:
# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [113]:
# 모델 학습
model.fit(x_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
2638/2638 [==============================] - 10s 2ms/step - loss: 1.2021 - accuracy: 0.3131
Epoch 2/10
2638/2638 [==============================] - 6s 2ms/step - loss: 0.4901 - accuracy: 0.3031
Epoch 3/10
2638/2638 [==============================] - 8s 3ms/step - loss: 0.4477 - accuracy: 0.3016
Epoch 4/10
2638/2638 [==============================] - 6s 2ms/step - loss: 0.4230 - accuracy: 0.3011
Epoch 5/10
2638/2638 [==============================] - 10s 4ms/step - loss: 0.4212 - accuracy: 0.3014
Epoch 6/10
2638/2638 [==============================] - 6s 2ms/step - loss: 0.4108 - accuracy: 0.3009
Epoch 7/10
2638/2638 [==============================] - 7s 3ms/step - loss: 0.4024 - accuracy: 0.3009
Epoch 8/10
2638/2638 [==============================] - 7s 3ms/step - loss: 0.3760 - accuracy: 0.3010
Epoch 9/10
2638/2638 [==============================] - 6s 2ms/step - loss: 0.3698 - accuracy: 0.3011
Epoch 10/10
2638/2638 [==============================] - 8s 3ms/step - loss: 0.3

In [114]:
# 모델 예측
predictions = model.predict(x_test)

541/541 [==============================] - 1s 2ms/step


In [116]:
predictions

array([[0.9049086 ],
       [0.77963006],
       [0.92300093],
       ...,
       [0.9397096 ],
       [0.77091575],
       [0.62798643]], dtype=float32)

In [117]:
# 제출 파일을 읽어옵니다.
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/범죄유형/sample_submission.csv') 

In [118]:
# 예측한 값을 TARGET 컬럼에 할당합니다.
submit['TARGET'] = predictions
submit.head()

,ID,TARGET
0,TEST_00000,0.904909
1,TEST_00001,0.779630
2,TEST_00002,0.923001
3,TEST_00003,0.906029
4,TEST_00004,0.842916
